In [67]:
import os
import pandas as pd
import numpy as np

In [97]:
# Reference: https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af
# Setting variables to be used in format string command below
tweet_count = 1000000
username = "stephenasmith"

# Using OS library to call CLI commands in Python
os.system("snscrape --jsonl --max-results {} twitter-search 'from:{}'> user-{}-tweets.json".format(tweet_count, username, username))

0

In [99]:
# Reads the json generated from the CLI command above and creates a pandas dataframe
tweets_df = pd.read_json('data/user-{}-tweets.json'.format(username), lines=True)

# Displays first 5 entries from dataframe
tweets_df.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/stephenasmith/status/14524...,2021-10-25 04:08:33+00:00,Don’t worry about that young fella @JaMorant. ...,Don’t worry about that young fella @JaMorant. ...,1452487216945639428,"{'_type': 'snscrape.modules.twitter.User', 'us...",216,1361,18327,...,None,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/stephenasmith/status/14523...,2021-10-24 19:14:17+00:00,Big thanks to @EBONYMag for naming me to the E...,Big thanks to @EBONYMag for naming me to the E...,1452352767293599750,"{'_type': 'snscrape.modules.twitter.User', 'us...",164,82,1373,...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[EBONYPower100],None
2,snscrape.modules.twitter.Tweet,https://twitter.com/stephenasmith/status/14519...,2021-10-23 16:41:55+00:00,Okay peeps. Okay peeps. Let’s crawl before we...,Okay peeps. Okay peeps. Let’s crawl before we...,1451952032890474506,"{'_type': 'snscrape.modules.twitter.User', 'us...",623,295,4456,...,None,NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,None,None,None,None,None,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/stephenasmith/status/14517...,2021-10-23 05:13:57+00:00,Props to @DwightHoward for addressing the issu...,Props to @DwightHoward for addressing the issu...,1451778901747945474,"{'_type': 'snscrape.modules.twitter.User', 'us...",271,274,5018,...,None,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/stephenasmith/status/14517...,2021-10-23 03:54:16+00:00,This is some embarrassing stuff being displaye...,This is some embarrassing stuff being displaye...,1451758850068074498,"{'_type': 'snscrape.modules.twitter.User', 'us...",852,1935,20484,...,None,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None


In [100]:
tweets_df.shape

(38350, 28)

In [101]:
arr = tweets_df.to_numpy()

In [6]:
def getPercentage(arr, textIndex, phrase):
    count = 0
    for val in arr:
        if phrase in val[textIndex].lower():
            count += 1
    return count * 100 / len(arr)

In [102]:
print(getPercentage(arr, 3, 'lebron'))

1.2490221642764017


In [103]:
# Check the earliest tweet in the dataset
arr[-1][2].year

2009

In [104]:
tweets_df.columns

Index(['_type', 'url', 'date', 'content', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags'],
      dtype='object')

In [105]:
parseDate = lambda x: [x.week, x.month, x.year]

def cleanData(data):
    # Extract user, date, content, replyCount, retweetCount, and likeCount
    data = data.loc[:, ['user', 'date', 'content', 'replyCount', 'retweetCount', 'likeCount']]

    # Extract username from user
    username = data['user'][0]['username']
    data['user'] = username

    # Convert date: [week, month, year]
    data['date'] = data['date'].apply(np.vectorize(parseDate))
    
    data.to_csv('data/user-{}-tweets-cleaned.csv'.format(username), index = False)

In [106]:
cleanData(tweets_df)